<a href="https://colab.research.google.com/github/SaurabhChadda/Portfolio-Projects/blob/main/Pyspark_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here in this step I'm connecting the google colab with my google drive as I'm importing data from there.

In [ ]:
from google.colab import drive
drive.mount("/content/mount")

Drive already mounted at /content/mount; to attempt to forcibly remount, call drive.mount("/content/mount", force_remount=True).


Here I'm installing the pyspark package.

In [ ]:
!pip install pyspark

Here in this step we are installing all the required packages.

In [ ]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

Here I'm importing a sparksession and creating session named as 'project'.

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('project').getOrCreate()

Importing data from the drive for which we have already created a mount to connect google colab with the drive.

In [ ]:
customer=spark.read.csv("/content/mount/MyDrive/Store data normalize/Customer Data.csv",header=True)
orders=spark.read.csv("/content/mount/MyDrive/Store data normalize/Order data.csv",header=True)
products=spark.read.csv("/content/mount/MyDrive/Store data normalize/Product Data.csv",header=True)
region=spark.read.csv("/content/mount/MyDrive/Store data normalize/Regional data.csv",header=True)
transactions=spark.read.csv("/content/mount/MyDrive/Store data normalize/Transaction data.csv",header=True)


 Let's check the details regarding the dataset so, that we can make any changes if needed.

In [ ]:
customer.printSchema()


root
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)



In [ ]:
orders.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)



In this step we are converting the order date and ship date from strings to date format so that we can do different analysis on the basis of that.

In [ ]:
orders=orders.withColumn('Order Date',to_date('Order Date','dd-mm-yyyy')).withColumn('Ship Date',to_date('Ship Date','dd-mm-yyyy'))

In [ ]:
products.printSchema()

root
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)



In [ ]:
region.printSchema()

root
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)



In [ ]:
transactions.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: string (nullable = true)



 In this step we are dividing the customers on the basis of the segment from which they belongs so,it will give us a clearer
where the business performing good and where we have to improve.

In [ ]:
customers_grouped=customer.groupBy('Segment').count().withColumnRenamed('count','Total')
customers_grouped.show()

+-----------+-----+
|    Segment|Total|
+-----------+-----+
|   Consumer|  409|
|Home Office|  148|
|  Corporate|  236|
+-----------+-----+



In this step we are trying to fecth the details regarding what is the percentage formed by each category of customers in the total customers count.

In [ ]:
customers_grouped.withColumn('Percent',round(customers_grouped['Total']/customer.count()*100,2)).show()

+-----------+-----+-------+
|    Segment|Total|Percent|
+-----------+-----+-------+
|   Consumer|  409|  51.58|
|Home Office|  148|  18.66|
|  Corporate|  236|  29.76|
+-----------+-----+-------+



 Here we are trying to fecth details regarding the most preffered shipping mode by the customers.

In [ ]:
Shipmode_analysis=orders.groupBy('Ship Mode').count().withColumnRenamed('count','Total')
Shipmode_analysis.show()

+--------------+-----+
|     Ship Mode|Total|
+--------------+-----+
|   First Class|  787|
|      Same Day|  264|
|  Second Class|  964|
|Standard Class| 2994|
+--------------+-----+



What percent of total shipping mode constituted by each of the shipping mode.

In [ ]:
Shipmode_analysis.withColumn('Percent',round(Shipmode_analysis['Total']/orders.count()*100,2)).show()

+--------------+-----+-------+
|     Ship Mode|Total|Percent|
+--------------+-----+-------+
|   First Class|  787|  15.71|
|      Same Day|  264|   5.27|
|  Second Class|  964|  19.25|
|Standard Class| 2994|  59.77|
+--------------+-----+-------+



 In this step I'm analysing that how many orders placed in each year and if there is any increase or not in the orders and sorting the data in ascending order

In [ ]:
Yearly_orders=orders.groupBy(year('Order Date').alias('Year')).count().withColumnRenamed('count','Total_orders')
Yearly_orders=Yearly_orders.sort(Yearly_orders['Year'].asc())
Yearly_orders.show()

+----+------------+
|Year|Total_orders|
+----+------------+
|2014|         969|
|2015|        1038|
|2016|        1315|
|2017|        1687|
+----+------------+



In this step we are trying to calculate the percentage change in total orders acount in each year.

In [ ]:
windowSpec=Window.partitionBy().orderBy(Yearly_orders['Year'])
Yearly_orders=Yearly_orders.withColumn("next_total",lead('Total_orders').over(windowSpec))
Yearly_orders=Yearly_orders.select('year','total_orders','next_total',(round((Yearly_orders['next_total']-Yearly_orders['Total_orders'])/Yearly_orders['Total_orders']*100,2)).
                     alias('Percent Change'))
Yearly_orders.show()


+----+------------+----------+--------------+
|year|total_orders|next_total|Percent Change|
+----+------------+----------+--------------+
|2014|         969|      1038|          7.12|
|2015|        1038|      1315|         26.69|
|2016|        1315|      1687|         28.29|
|2017|        1687|      NULL|          NULL|
+----+------------+----------+--------------+



In this step we are dividing our total orders on the basis product categories to get the indepth information about the orders.

In [ ]:
Product_category=products.groupBy('Category').count().withColumnRenamed('count','Total_products')
Product_category.withColumn('Total_percent',round(Product_category['Total_products']/products.count()*100,2)).show()

+---------------+--------------+-------------+
|       Category|Total_products|Total_percent|
+---------------+--------------+-------------+
|Office Supplies|          6026|         60.3|
|      Furniture|          2121|        21.22|
|     Technology|          1847|        18.48|
+---------------+--------------+-------------+



In this step we are dividing our total orders on the basis product subcategories to get the indepth information about the orders.

In [ ]:
subcategory_analysis=products.groupBy('Sub-Category').count().withColumnRenamed('count','Total_product')
subcategory_analysis=subcategory_analysis.withColumn('Total_percent',round(subcategory_analysis['Total_product']/products.count()*100,2))
subcategory_analysis.sort(subcategory_analysis['Total_percent'].desc()).show()

+------------+-------------+-------------+
|Sub-Category|Total_product|Total_percent|
+------------+-------------+-------------+
|     Binders|         1523|        15.24|
|       Paper|         1370|        13.71|
| Furnishings|          957|         9.58|
|      Phones|          889|          8.9|
|     Storage|          846|         8.47|
|         Art|          796|         7.96|
| Accessories|          775|         7.75|
|      Chairs|          617|         6.17|
|  Appliances|          466|         4.66|
|      Labels|          364|         3.64|
|      Tables|          319|         3.19|
|   Envelopes|          254|         2.54|
|   Bookcases|          228|         2.28|
|   Fasteners|          217|         2.17|
|    Supplies|          190|          1.9|
|    Machines|          115|         1.15|
|     Copiers|           68|         0.68|
+------------+-------------+-------------+



In this step we are trying fetch details regarding from which region we have the maximum numbers of customers and where we have to channelise more support.

In [ ]:
region.groupBy('Region').count().show()

+-------+-----+
| Region|count|
+-------+-----+
|  South| 1620|
|Central| 2323|
|   East| 2848|
|   West| 3203|
+-------+-----+



In [ ]:
transactions.show(3)

+--------------+-----------+---------------+------+--------+--------+-------+
|      Order ID|Customer ID|     Product ID| Sales|Quantity|Discount| Profit|
+--------------+-----------+---------------+------+--------+--------+-------+
|CA-2016-152156|   CG-12520|FUR-BO-10001798|261.96|       2|       0|41.9136|
|CA-2016-152156|   CG-12520|FUR-CH-10000454|731.94|       3|       0|219.582|
|CA-2016-138688|   DV-13045|OFF-LA-10000240| 14.62|       2|       0| 6.8714|
+--------------+-----------+---------------+------+--------+--------+-------+
only showing top 3 rows



In [ ]:
orders.show(3)

+--------------+----------+----------+--------------+-----------+
|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|
+--------------+----------+----------+--------------+-----------+
|CA-2016-152156|2016-01-08|2016-01-11|  Second Class|   CG-12520|
|CA-2016-138688|2016-01-12|2016-01-16|  Second Class|   DV-13045|
|US-2015-108966|2015-01-11|2015-01-18|Standard Class|   SO-20335|
+--------------+----------+----------+--------------+-----------+
only showing top 3 rows



In this step we are performing a left join between orders and transactions dataframe to get the key information which supports are analysis for the sales and profits.

In [ ]:
Sales_analysis=transactions.join(orders,transactions['Order ID']==orders['Order ID'],'left').select(transactions['Order ID'],transactions['Customer ID'],orders['Order Date'],
                                                                                     transactions['Product ID'],orders['Ship Mode'],transactions['Sales'],
                                                                                     transactions['Quantity'],transactions['Discount'])
Sales_analysis.show(5)

+--------------+-----------+----------+---------------+--------------+--------+--------+--------+
|      Order ID|Customer ID|Order Date|     Product ID|     Ship Mode|   Sales|Quantity|Discount|
+--------------+-----------+----------+---------------+--------------+--------+--------+--------+
|CA-2016-152156|   CG-12520|2016-01-08|FUR-BO-10001798|  Second Class|  261.96|       2|       0|
|CA-2016-152156|   CG-12520|2016-01-08|FUR-CH-10000454|  Second Class|  731.94|       3|       0|
|CA-2016-138688|   DV-13045|2016-01-12|OFF-LA-10000240|  Second Class|   14.62|       2|       0|
|US-2015-108966|   SO-20335|2015-01-11|FUR-TA-10000577|Standard Class|957.5775|       5|    0.45|
|US-2015-108966|   SO-20335|2015-01-11|OFF-ST-10000760|Standard Class|  22.368|       2|     0.2|
+--------------+-----------+----------+---------------+--------------+--------+--------+--------+
only showing top 5 rows



 Here in this step I'm creating a new column named as total profit which is the product of sales price and quantity sold.

In [ ]:
Sales_analysis=Sales_analysis.withColumn('Total_profit',Sales_analysis['sales'] * Sales_analysis['quantity'])


Let's try to fetch the product id of top 10 most profitable products.

In [ ]:
Revenue_analysis=Sales_analysis.groupBy('product id').sum('total_profit').withColumnRenamed('sum(total_profit)','Total Revenue')
Top_profit=Revenue_analysis.join(products,Revenue_analysis['product id']==products['product id'],'left').select(Revenue_analysis['product id'],Revenue_analysis['Total Revenue'],
                                                                                                     products['category'],products['Sub-Category'],products['Product Name'])
Top_profit.sort(Top_profit['Total Revenue'].desc()).show(10,truncate=False)


+---------------+-------------+---------------+------------+------------------------------------------------+
|product id     |Total Revenue|category       |Sub-Category|Product Name                                    |
+---------------+-------------+---------------+------------+------------------------------------------------+
|TEC-CO-10004722|253399.276   |Technology     |Copiers     |Canon imageCLASS 2200 Advanced Copier           |
|TEC-CO-10004722|253399.276   |Technology     |Copiers     |Canon imageCLASS 2200 Advanced Copier           |
|TEC-CO-10004722|253399.276   |Technology     |Copiers     |Canon imageCLASS 2200 Advanced Copier           |
|TEC-CO-10004722|253399.276   |Technology     |Copiers     |Canon imageCLASS 2200 Advanced Copier           |
|TEC-CO-10004722|253399.276   |Technology     |Copiers     |Canon imageCLASS 2200 Advanced Copier           |
|OFF-BI-10000545|171524.892   |Office Supplies|Binders     |GBC Ibimaster 500 Manual ProClick Binding System|
|OFF-BI-10

Let's try to fetch the names of the least profit generating products in all these years

In [ ]:
Top_profit.sort(Top_profit['Total Revenue'].asc()).show(10,truncate=False)

+---------------+-------------+---------------+------------+--------------------------------------------------------------------------+
|product id     |Total Revenue|category       |Sub-Category|Product Name                                                              |
+---------------+-------------+---------------+------------+--------------------------------------------------------------------------+
|OFF-AP-10002203|3.248        |Office Supplies|Appliances  |Eureka Disposable Bags for Sanitaire Vibra Groomer I Upright Vac          |
|OFF-PA-10000048|6.48         |Office Supplies|Paper       |Xerox 20                                                                  |
|OFF-LA-10003388|11.52        |Office Supplies|Labels      |Avery 5                                                                   |
|OFF-EN-10001535|14.144       |Office Supplies|Envelopes   |Grip Seal Envelopes                                                       |
|OFF-AR-10002704|14.98        |Office Supplies|A

Here in this step we are trying to calculate the percentage change in profits in all these years.

In [ ]:
Profit_percent=Sales_analysis.groupBy(year('Order Date').alias('Year')).sum('Total_profit').withColumnRenamed('sum(Total_profit)','Total Revenue')
Profit_percent=Profit_percent.select('Year',round('Total Revenue',2).alias('Total Revenue')).sort(Profit_percent['Year'].asc())
windowspec1=Window.partitionBy().orderBy(Profit_percent['Year'])
profit_percentage_analysis=Profit_percent.withColumn('Next_year_revenue',lead('Total Revenue').over(windowspec1))
profit_percentage_analysis.select('Year','Total Revenue','Next_year_revenue',((profit_percentage_analysis['Next_year_revenue']-profit_percentage_analysis['Total Revenue'])/
                                  profit_percentage_analysis['Total Revenue']*100)).show()

In [ ]:
profit_percentage_analysis=profit_percentage_analysis.select('Year','Total Revenue','Next_year_revenue').withColumn('Percentage_change',
 (profit_percentage_analysis['Next_year_revenue']-profit_percentage_analysis['Total Revenue'])/
                                  profit_percentage_analysis['Total Revenue']*100)
profit_percentage_analysis.show()

+----+-------------+-----------------+------------------+
|Year|Total Revenue|Next_year_revenue| Percentage_change|
+----+-------------+-----------------+------------------+
|2014|   2468283.73|       2384901.38|-3.378150938911715|
|2015|   2384901.38|       3050057.36| 27.89029288917599|
|2016|   3050057.36|        3584819.6|17.532858464012634|
|2017|    3584819.6|             NULL|              NULL|
+----+-------------+-----------------+------------------+

